In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import random
import copy
import ast
import pprint
import json
import re
import time

/tmp/ipykernel_2651022/3939713480.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def set_seed():
    random.seed(0)
    np.random.seed(0)

In [3]:
GENERATE_METHODS_DIR = Path('data/docs/manual')
METHODS_DIR = Path('data/docs/methods')
PROMPT_SEEDS_DIR = Path('data/prompts/generation/components')
PROMPT_COMPONENTS_DIR = Path('data/prompts/generation/components')
GEN_PROMPTS_DIR = Path('data/prompts/generation/output')
VAL_PROMPTS_DIR = Path('data/prompts/validation/output')
DATA_DIR = Path('data/')

# device types & locations

In [4]:
DEVICES = list(pd.read_csv(DATA_DIR/'datasets/partial/device_types.csv')['type'])
DEVICES

['Cover', 'Switch', 'Light', 'Input', 'Temperature', 'Smoke', 'Humidity']

In [5]:
DEVICE_LOCATIONS = pd.read_csv(DATA_DIR/'datasets/partial/device_locations.csv')
DEVICE_LOCATIONS

,device,location
0,Cover,Bedroom
1,Cover,Living room
2,Cover,Kitchen
3,Cover,Bathroom
4,Cover,Patio
...,...,...
65,Humidity,Basement
66,Humidity,Laundry room
67,Humidity,Garage
68,Humidity,Attic


# seeds generation

In [13]:
target_device = devices[2]

with open('prompts/prompt.md', "w") as f:
    f.write(f'')

with open('prompts/instruction_onedevice.md', "r") as f:
    instruction = f.read()
with open('prompts/seeds_multiapi.md', "r") as f:
    seeds = f.read()

with open('prompts/prompt.md', "a") as f:
    f.write(f'{instruction}\n\n'
            f'Example:\n\n{seeds}\n\n'
            f'For generation: \n\n'
            f'Device: {target_device} id=XXX\n\n')

row_i = 0
for p in Path('docs/').iterdir():
    if target_device in p.name and p.name != f'{target_device}.md':
        with open(p, "r") as f:
            method_str = f.read()
        with open('prompts/prompt.md', "a") as f:
            f.write(f'API method {row_i}: {method_str}\n')
        row_i += 1
    if row_i == 5:
        break

# with open('prompts/prompt.md', "a") as f:
#     f.write(f'User command 0:')

In [49]:
output = """[{"User command 0.1": "Reset the energy counter for the main light.", "JSON command 0.1": {"method": "Light.ResetCounters", "params": {"id": "id", "type": ["energy"]}}}, {"User command 0.2": "Reset the all counter for the light, please.", "JSON command 0.2": {"method": "Light.ResetCounters", "params": {"id": "id"}}}, {"User command 1.1" : "Could you please fetch the configuration details for the living room light, including its name, night mode settings, and transition duration?", "JSON command 1.1": {"method": "Light.GetConfig", "params": {"id": "id"}}}, {"User command 1.2" : "Tell me, what is transition duration for the main light?", "JSON command 1.2": {"method": "Light.GetConfig", "params": {"id": "id"}}}]"""
output = output.replace("XXX", '"id"')

In [50]:
output = re.sub(r'(User command \d\.\d)', 'User command', output)
output = re.sub(r'(JSON command \d\.\d)', 'JSON command', output)

In [51]:
print(output)

[{"User command": "Reset the energy counter for the main light.", "JSON command": {"method": "Light.ResetCounters", "params": {"id": "id", "type": ["energy"]}}}, {"User command": "Reset the all counter for the light, please.", "JSON command": {"method": "Light.ResetCounters", "params": {"id": "id"}}}, {"User command" : "Could you please fetch the configuration details for the living room light, including its name, night mode settings, and transition duration?", "JSON command": {"method": "Light.GetConfig", "params": {"id": "id"}}}, {"User command" : "Tell me, what is transition duration for the main light?", "JSON command": {"method": "Light.GetConfig", "params": {"id": "id"}}}]


In [52]:
import json

In [53]:
cmds = json.loads(output)

In [54]:
json.dumps(cmds)

'[{"User command": "Reset the energy counter for the main light.", "JSON command": {"method": "Light.ResetCounters", "params": {"id": "id", "type": ["energy"]}}}, {"User command": "Reset the all counter for the light, please.", "JSON command": {"method": "Light.ResetCounters", "params": {"id": "id"}}}, {"User command": "Could you please fetch the configuration details for the living room light, including its name, night mode settings, and transition duration?", "JSON command": {"method": "Light.GetConfig", "params": {"id": "id"}}}, {"User command": "Tell me, what is transition duration for the main light?", "JSON command": {"method": "Light.GetConfig", "params": {"id": "id"}}}]'

In [55]:
df_dict = {'device': [], 'user_cmd': [], 'used_mtd': [], 'json_cmd': []}
for row_i in range(len(cmds)):
    df_dict['user_cmd'].append(cmds[row_i][f'User command'])
    df_dict['json_cmd'].append(cmds[row_i][f'JSON command'])
    df_dict['used_mtd'].append(f'{cmds[row_i][f"JSON command"]["method"]}')
    df_dict['device'].append(target_device)
print(df_dict)

{'device': ['Light', 'Light', 'Light', 'Light'], 'user_cmd': ['Reset the energy counter for the main light.', 'Reset the all counter for the light, please.', 'Could you please fetch the configuration details for the living room light, including its name, night mode settings, and transition duration?', 'Tell me, what is transition duration for the main light?'], 'used_mtd': ['Light.ResetCounters', 'Light.ResetCounters', 'Light.GetConfig', 'Light.GetConfig'], 'json_cmd': [{'method': 'Light.ResetCounters', 'params': {'id': 'id', 'type': ['energy']}}, {'method': 'Light.ResetCounters', 'params': {'id': 'id'}}, {'method': 'Light.GetConfig', 'params': {'id': 'id'}}, {'method': 'Light.GetConfig', 'params': {'id': 'id'}}]}


In [56]:
data_path = Path('data/')

In [57]:

df = pd.DataFrame(df_dict)
df.to_csv(data_path / f'prompt_seeds.csv', index=False, mode='a', header=False)

# prompt generation

In [14]:
import pprint

In [15]:
device_methods_dict = {device: {'basic': [], 'advance': []} for device in devices}

advance_methods = ['GetConfig', 'SetConfig', 'GetStatus']
for p in METHODS_DIR.iterdir():
    for d in devices:
        if d in p.name:
            if p.stem.split('.')[1] in advance_methods:
                device_methods_dict[d]['advance'].append(p.stem)
            else:
                device_methods_dict[d]['basic'].append(p.stem)

# device_methods_dict['Switch']['basic'].append('Switch.Toggle')
# device_methods_dict['Light']['basic'].append('Light.ResetCounters')

# device_methods_dict['Temperature']['advance'].append('Temperature.GetStatus')
# device_methods_dict['Switch']['advance'].append('Switch.GetStatus')
# device_methods_dict['Smoke']['advance'].append('Smoke.SetConfig')
# device_methods_dict['Temperature']['advance'].append('Temperature.SetConfig')
# device_methods_dict['Switch']['advance'].append('Switch.GetConfig')
# device_methods_dict['Cover']['advance'].append('Cover.SetConfig')

pprint.pprint(device_methods_dict)

{'Cover': {'advance': ['Cover.GetConfig', 'Cover.GetStatus', 'Cover.SetConfig'],
           'basic': ['Cover.Close',
                     'Cover.Open',
                     'Cover.GoToPosition',
                     'Cover.Calibrate',
                     'Cover.ResetCounters',
                     'Cover.Stop']},
 'Humidity': {'advance': ['Humidity.SetConfig',
                          'Humidity.GetConfig',
                          'Humidity.GetStatus'],
              'basic': ['Humidity.SetHumidity']},
 'Input': {'advance': ['Input.GetConfig', 'Input.SetConfig', 'Input.GetStatus'],
           'basic': ['Input.ResetCounters']},
 'Light': {'advance': ['Light.GetConfig', 'Light.SetConfig', 'Light.GetStatus'],
           'basic': ['Light.ResetCounters',
                     'Light.Toggle',
                     'Light.Calibrate',
                     'Light.Set']},
 'Smoke': {'advance': ['Smoke.GetConfig', 'Smoke.GetStatus', 'Smoke.SetConfig'],
           'basic': ['Smoke.Mute']},
 'Swit

In [16]:
def generate_seed(seeds_df, num_methods=2, num_examples=2):
    rand_device = random.choice(list(seeds_df['device'].unique()))
    seed_methods = random.sample(list(seeds_df[seeds_df['device'] == rand_device]['used_mtd'].unique()), num_methods)
    methods_description = ''
    cmds = {}
    for method_i, method in enumerate(seed_methods):
        with open(METHODS_DIR / f'{method}.md', "r") as f:
            method_str = f.read()
        methods_description += f'API method {method_i+1}: {method_str}\n'
        example_ids = random.sample(list(seeds_df[seeds_df['used_mtd'] == method].index), num_examples)
        for example_i, row_i in enumerate(example_ids):
            cmds[f'Example {method_i+1}.{example_i+1}'] = {f'User command': seeds_df.loc[row_i, 'user_cmd'],
                 f'JSON command': ast.literal_eval(seeds_df.loc[row_i, 'json_cmd'].replace("'", '"'))}
    seed_description = (
        f'The next device, methods and JSON are example:\n\n'
        f'Device: {rand_device} id=33\n\n'
        f'{methods_description}'
        f'{json.dumps({"commands": cmds})}'
    )
    return seed_description

def generate_advance_seed(seeds_df):
    rand_device = random.choice(list(seeds_df['device'].unique()))
    method = random.choice(list(seeds_df[seeds_df['device'] == rand_device]['used_mtd'].unique()))
    cmds = {}
    with open(METHODS_DIR / f'{method}.md', "r") as f:
        method_str = f.read()
    method_description = f'API method 1:\n{method_str}\n'
    sample_method_ids = list(seeds_df[seeds_df['used_mtd'] == method].index)
    example_ids = random.sample(sample_method_ids, min(4, len(sample_method_ids)))
    for example_i, row_i in enumerate(example_ids):
        cmds[f'Example {example_i+1}'] = {f'User command': seeds_df.loc[row_i, 'user_cmd'],
            f'JSON command': ast.literal_eval(seeds_df.loc[row_i, 'json_cmd'].replace("'", '"'))}
    seed_description = (
        f'The next device, methods and JSON are example:\n\n'
        f'Device: {rand_device} id=33\n\n'
        f'{method_description}'
        f'{json.dumps({"commands": cmds})}'
    )
    return seed_description

In [17]:
seeds_df = pd.read_csv(PROMPT_SEEDS_DIR / 'prompt_seeds.csv')
# json.loads(seeds_df.loc[0, 'json_cmd'].replace("'", '"'))['method']
# f"dict: {json.dumps(ast.literal_eval(seeds_df.loc[0, 'json_cmd']))}"
print(generate_seed(seeds_df, num_examples=3))

The next device, methods and JSON are example:

Device: Cover id=33

API method 1: Method name: Cover.Close
Method description:
Properties:
{"id": {"type": "number", "description": "The numeric ID of the Cover component instance"}, "duration": {"type": "number", "description": "If duration is not provided, Cover will fully close, unless it times out because of maxtime_close first. If duration (seconds) is provided, Cover will move in the close direction for the specified time. duration must be in the range [0.1..maxtime_open]Optional"}}
Response:
null on success; error if the request can not be executed or failed

API method 2: Method name: Cover.Open
Method description: 
Properties:
{"id": {"type": "number", "description": "The numeric ID of the Cover component instance"}, "duration": {"type": "number", "description": "If duration is not provided, Cover will fully open, unless it times out because of maxtime_open first. If duration (seconds) is provided, Cover will move in the open di

In [18]:
NUM_METHODS_GEN = 2
NUM_METHODS_SEED = 2
NUM_EXAMPLES_SEED = 3
EXAMPLES_BASIC_METHOD = NUM_EXAMPLES_SEED
EXAMPLES_ADVANCE_METHOD = 8

In [19]:
seeds_df = pd.read_csv(PROMPT_SEEDS_DIR / 'prompt_seeds.csv')

gen_prompts_df = pd.DataFrame(columns=['type', 'device', 'last_mtd_i', 'text'])
for device, methods in device_methods_dict.items():
    method_i = 0
    methods_description = ''

    for i, method_name in enumerate(methods['basic']):
        with open(METHODS_DIR / f'{method_name}.md', "r") as f:
            method_str = f.read()
        methods_description += f'API method {method_i+1}: {method_str}\n'
        method_i += 1

        if (method_i % NUM_METHODS_GEN == 0 and method_i != 0) or i == len(methods['basic']) - 1:
            with open(PROMPT_COMPONENTS_DIR / 'instruction_onedevice.md', "r") as f:
                instruction = f.read()
            instruction = instruction.replace('{NUM_EXAMPLE_COMMANDS}', str(method_i*EXAMPLES_BASIC_METHOD))
            instruction = instruction.replace('{EXAMPLES_BASIC_METHOD}', str(EXAMPLES_BASIC_METHOD))

            seed_1 = generate_seed(seeds_df, NUM_METHODS_SEED, NUM_EXAMPLES_SEED)
            prompt_text = (f'{instruction}\n\n'
                        f'{seed_1}\n\n'
                        f'The next device and methods are for you to generate commands:\n\n'
                        f'Device: {device} id=444\n\n'
                        f'{methods_description}')
            gen_prompts_df.loc[len(gen_prompts_df)] = pd.Series({'type': 'basic',
        'device': device,
        'last_mtd_i': i,
        'text': prompt_text})

            method_i = 0
            methods_description = ''
    
    for i, method_name in enumerate(methods['advance']):
        with open(METHODS_DIR / f'{method_name}.md', "r") as f:
            method_str = f.read()
        method_description = f'API method 1:\n{method_str}\n'

        with open(PROMPT_COMPONENTS_DIR / 'instruction_advance.md', "r") as f:
            instruction = f.read()
        instruction = instruction.replace('{NUM_EXAMPLE_COMMANDS}', str(EXAMPLES_ADVANCE_METHOD))

        seed_1 = generate_advance_seed(seeds_df)
        
        prompt_text = (f'{instruction}\n\n'
                    f'{seed_1}\n\n'
                    f'The next device and method are for you to generate commands:\n\n'
                    f'Device: {device} id=444\n\n'
                    f'{method_description}')
        gen_prompts_df.loc[len(gen_prompts_df)] = pd.Series({'type': 'advance',
        'device': device,
        'last_mtd_i': i,
        'text': prompt_text})

gen_prompts_df.to_csv(GEN_PROMPTS_DIR / 'prompts.csv', index=False)

# inspection

In [22]:
def parse_output(output):
    output = json.loads(output)['commands']
    df_dict = {'device': [], 'user_cmd': [], 'mtd': [], 'json_cmd': []}
    for cmds in list(output.values()):
        df_dict['user_cmd'].append(cmds[f'User command'])
        df_dict['json_cmd'].append(json.dumps(cmds[f'JSON command']))
        df_dict['mtd'].append(f'{cmds[f"JSON command"]["method"]}')
        df_dict['device'].append(f'{cmds[f"JSON command"]["method"].split(".")[0]}')
    df = pd.DataFrame(df_dict)
    header = not (VAL_PROMPTS_DIR / 'prompts.csv').exists()
    df.to_csv(VAL_PROMPTS_DIR / 'prompts.csv', index=False, mode='a', header=header)

# Dataset generation

In [23]:
from openai import OpenAI
with open('data/keys/openai.txt') as f:
    key = f.read()
client = OpenAI(api_key=key)

In [24]:
gen_prompts_df = pd.read_csv(GEN_PROMPTS_DIR / 'prompts.csv')

parsed = []
for _, r in gen_prompts_df.iterrows():
    prompt = r['text']

    print(prompt)
    print('---------------------------------------')

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        response_format={"type": "json_object"},
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    parse_output(completion.choices[0].message.content)

    print(f'{r["device"]}:{r["last_mtd_i"]} parsed!\n\n')

    print(f'{completion.choices[0].message.content}\n\n')
    print('---------------------------------------')

    parsed.append(f'{r["device"]}:{r["last_mtd_i"]}')

    time.sleep(10)

You are provided with a device in a smart home, its API methods, their descriptions and required parameters. Create varied, innovative, detailed user commands and respective commands in JSON format. User commands must sound naturally. Command to device must be strictly in JSON format. Generate exactly 6 example commands. For each API method generate 3 commands that use this method. You mustn't hallucinate new API methods. In each pair of user command with JSON command use at least half of possible parametrs. Ask about parameters from reponse output if they exist. Don't add to JSON command parameters that are not mentioned in the user command. Output must be strictly JSON array: {"commands": {"Example 1.1": {"User command": "...", "JSON command": {...}}, "Example 1.2": {"User command": "...", "JSON command": {...}}, "Example 1.3": {"User command": "...", "JSON command": {...}}, ...}}. There is example below.


The next device, methods and JSON are example:

Device: Cover id=33

API meth

# Filter methods parameters

In [5]:
a = {0: {1: 1}}
b = a[0]
b[1] = 2
print(a, b)

{0: {1: 2}} {1: 2}


In [27]:
def remove_hallucinated_parameters(gt_json, json_scheme):
    json_temp = gt_json.copy()
    for key, val in json_temp.items():
        if key not in json_scheme:
            gt_json.pop(key, None)
            print(f'hallucinated: {key}')
            continue
        if isinstance(val, dict):
            remove_hallucinated_parameters(val, json_scheme[key]['properties'])

In [20]:
gt_json = {0: {1: 1}, 1: {1: 1}}
json_scheme = {1: {'properties': {1: 1}}}
remove_hallucinated_parameters(gt_json, json_scheme)
gt_json

{1: {1: 1}}

In [30]:
json_schemes_df = pd.read_csv(METHODS_DIR.parent / 'methods_json.csv')
df = pd.read_csv(VAL_PROMPTS_DIR / 'prompts.csv')

for i, row in df.iterrows():
    method_name = row['mtd']
    method_df = json_schemes_df[json_schemes_df['method'] == method_name]
    if method_df.shape[0] == 0:
        print(f'Incorrect method name in: {i}')
        continue
    json_scheme = json.loads(method_df.iloc[0]['json'])
    print(json_scheme)
    json_cmd = json.loads(row['json_cmd'])
    print(json_cmd)
    remove_hallucinated_parameters(json_cmd, json_scheme)
    print('<<<---------------------------->>>')    
    row['json_cmd'] = json.dumps(json_cmd)

df.to_csv(VAL_PROMPTS_DIR / 'prompts_filtered.csv', index=False)

{'method': 'Cover.Close', 'params': {'type': 'object', 'properties': {'id': {'type': 'number', 'description': 'The numeric ID of the Cover component instance'}, 'duration': {'type': 'number', 'description': 'If duration is not provided, Cover will fully close, unless it times out because of maxtime_close first. If duration (seconds) is provided, Cover will move in the close direction for the specified time. duration must be in the range [0.1..maxtime_open]Optional'}}}}
{'method': 'Cover.Close', 'params': {'id': 444, 'duration': 20}}
<<<---------------------------->>>
{'method': 'Cover.Close', 'params': {'type': 'object', 'properties': {'id': {'type': 'number', 'description': 'The numeric ID of the Cover component instance'}, 'duration': {'type': 'number', 'description': 'If duration is not provided, Cover will fully close, unless it times out because of maxtime_close first. If duration (seconds) is provided, Cover will move in the close direction for the specified time. duration must b

# Merge dataset

In [32]:
dataset_df = pd.read_csv(DATASET_PATH)
generated_df = pd.read_csv(VAL_PROMPTS_DIR / 'prompts_filtered.csv')

last_id = dataset_df.iloc[-1]['id']
generated_df['id'] = pd.Series(list(generated_df.index)) + last_id

new_df = pd.concat([dataset_df, generated_df], ignore_index=True, axis=0)
new_df.to_csv('data/datasets/dataset_v5.csv', index=False)

# Add DEVICE_NAME

In [4]:
from openai import OpenAI
with open('data/keys/openai.txt') as f:
    key = f.read()
client = OpenAI(api_key=key)

In [5]:
def parse_output(output, rows, output_path):
    output = json.loads(output)
    df_dict = {'id': [], 'device': [], 'user_cmd': [], 'mtd': [], 'json_cmd': []}
    for i in range(len(rows)):
        df_dict['user_cmd'].append(output[f'Edited command {i+1}'])
        df_dict['json_cmd'].append(rows[i]['json_cmd'])
        df_dict['mtd'].append(rows[i]['mtd'])
        df_dict['device'].append(rows[i]['device'])
        df_dict['id'].append(rows[i]['id'])
    df = pd.DataFrame(df_dict)
    header = not (output_path).exists()
    df.to_csv(output_path, index=False, mode='a', header=header)

In [6]:
df = pd.read_csv(DATASET_PATH)

with open(PROMPT_COMPONENTS_DIR / 'instruction_device_name.md') as f:
    instruction = f.read()

output_path = Path('data/datasets/dataset_v6.csv')

devices = []
commands = []
rows = []
num_send_commands = 6
for i in range(60, df.shape[0]): # df.shape[0]
    device = df.iloc[i]['device']
    user_cmd = df.iloc[i]['user_cmd']

    devices.append(device)
    commands.append(user_cmd)
    rows.append(df.iloc[i])

    if (len(devices) % 6 == 0 and len(devices) != 0) or i == df.shape[0] - 1:
        prompt = [f'Device type {i+1}: {device_type}\nUser command {i+1}: {command}'
                  for i, (device_type, command) in enumerate(zip(devices, commands))]
        prompt = '\n\n'.join(prompt)
        prompt = (f'{instruction}\n\n'
                  f'{prompt}')

        print(prompt)
        print('---------------------------------------')

        completion = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            response_format={"type": "json_object"},
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        parse_output(completion.choices[0].message.content, rows, output_path)

        print(f'{completion.choices[0].message.content}\n\n')
        print('---------------------------------------')

        devices = []
        commands = []
        rows = []

        time.sleep(10)

You are provided with the device type and the user command. Firstly, remove any mentions of the place where device is located. Then, replace phrase that mentions device in the user command with a string {{DEVICE_NAME}}. Output must be strictly JSON object: {"Edited command 1": "...", "Edited command 2": "...", "Edited command 3": "..."}. Don't output anything else. There is an example below.

Device type 1: Humidity
User command 1: Update me on the humidity in the lounge.

Device type 2: Smoke
User command 2: Set the name of the smoke sensor to 'Office' now.

Device type 3: Temperature
User command 3: Set the temperature report threshold of the kitchen thermometer to 1C.

Device type 4: Input
User command 4: Tell me the status of the input in the guest room, please.

Device type 5: Light
User command 5: Disable auto on function for the office light.

Device type 6: Switch
User command 6: Check the instantaneous active power delivered by the switch in the bedroom.

{"Edited command 1": 

## Locations

In [36]:
# locations = {"Cover": ["Bedroom", "Living room", "Kitchen", "Bathroom", "Patio", "Garage", "Office", "Basement", "Attic", "Outdoor"], "Switch": ["Living room", "Bedroom", "Kitchen", "Dining area", "Bathroom", "Garage", "Outdoor", "Home office", "Hallway", "Entryway"], "Light": ["Living room", "Bedroom", "Kitchen", "Dining area", "Bathroom", "Outdoor", "Study room", "Basement", "Attic", "Closet"], "Input": ["Living room", "Bedroom", "Kitchen", "Home office", "Bathroom", "Garage", "Outdoor", "Study room", "Basement", "Attic"], "Temperature": ["Living room", "Bedroom", "Kitchen", "Bathroom", "Basement", "Attic", "Greenhouse", "Outdoor", "Wine cellar", "HVAC room"], "Smoke": ["Kitchen", "Living room", "Bedroom", "Garage", "Office", "Basement", "Attic", "Hallway", "Entryway", "Laundry room"], "Humidity": ["Bathroom", "Kitchen", "Living room", "Bedroom", "Greenhouse", "Basement", "Laundry room", "Garage", "Attic", "Plant nursery"]}

In [37]:
# df_dict = {'device': [], 'location': []}
# for device, places in locations.items():
#     df_dict['device'] = df_dict['device'] + [device] * len(places)
#     df_dict['location'] = df_dict['location'] + places
# df = pd.DataFrame(data=df_dict)
# df.to_csv('data/datasets/partial/locations.csv', index=False)

## Generated merged dataset

In [21]:
import random

In [22]:
def generate_env(target_type, target_name):
    possible_ids = list(range(1, 99))

    target_id = random.choice(possible_ids)
    possible_ids.remove(target_id)

    env_devices = [f'{target_type}, name="{target_name}", id={target_id}']

    generated_device_names = [target_name]
    num_devices = np.random.randint(3, 8)
    for i in range(num_devices - 1):
        device_name = target_name
        while device_name in generated_device_names:
            device_type = random.choice(DEVICES)
            location = random.choice(list(DEVICE_LOCATIONS[DEVICE_LOCATIONS['device'] == device_type]['location']))
            number = np.random.randint(1, 5)
            device_name = f'{location} {device_type} {number}'
        device_id = random.choice(possible_ids)

        env_devices.append(f'{device_type}, name="{device_name}", id={device_id}')

        generated_device_names.append(device_name)
        possible_ids.remove(device_id)
        
    random.shuffle(env_devices)
    env = ';\n'.join(env_devices)

    return env, target_id

In [23]:
set_seed()

templated_actions = pd.read_csv(DATA_DIR / 'datasets/partial/templated_actions_test.csv')
out_df = pd.DataFrame(columns=['id','device','location','number','device_name','device_id','env','user_cmd','mtd','json_cmd'])

SAMPLES_PER_TEMPLATE = 4

row_id = 0
for i, row in templated_actions.iterrows():
    for _ in range(SAMPLES_PER_TEMPLATE):
        device = row['device']

        location = random.choice(list(DEVICE_LOCATIONS[DEVICE_LOCATIONS['device'] == device]['location']))
        number = np.random.randint(1, 5)
        device_name = f'{location} {device} {number}'
        
        if not '{DEVICE_NAME}' in row['user_cmd']:
            print(i)
        user_cmd = row['user_cmd'].format(**{'DEVICE_NAME': device_name})

        env, device_id = generate_env(device, device_name)

        json_cmd = row['json_cmd']
        json_cmd = json_cmd.replace('444', str(device_id))

        sample_dict = {
            'id': row_id,
            'device': row['device'],
            'location': location,
            'number': number,
            'device_name': device_name,
            'device_id': device_id,
            'env': env,
            'user_cmd': user_cmd,
            'mtd': row['mtd'],
            'json_cmd': json_cmd
        }
        out_df.loc[len(out_df)] = pd.Series(sample_dict)

        row_id += 1

out_df.to_csv(DATA_DIR / 'datasets/merged/test_0.csv', index=False)

In [8]:
# templated_actions = pd.read_csv(DATA_DIR / 'datasets/partial/templated_actions.csv')
# templated_actions = templated_actions.drop(columns='id')
# templated_actions.to_csv(DATA_DIR / 'datasets/partial/templated_actions.csv', index=False)

## Split dataset

In [9]:
templated_actions = pd.read_csv(DATA_DIR / 'datasets/partial/templated_actions.csv')

In [20]:
set_seed()

train_idx = int(len(templated_actions) * 0.6)
val_idx = int(len(templated_actions) * 0.8)

df = templated_actions.sample(frac=1)
train_df = df.iloc[:train_idx]
val_df = df.iloc[train_idx:val_idx]
test_df = df.iloc[val_idx:]

train_df.to_csv(DATA_DIR / 'datasets/partial/templated_actions_train.csv', index=False)
val_df.to_csv(DATA_DIR / 'datasets/partial/templated_actions_val.csv', index=False)
test_df.to_csv(DATA_DIR / 'datasets/partial/templated_actions_test.csv', index=False)

In [18]:
test_df['mtd'].unique()

array(['Temperature.GetConfig', 'Humidity.GetStatus', 'Switch.GetStatus',
       'Light.SetConfig', 'Input.GetStatus', 'Smoke.Mute', 'Cover.Open',
       'Switch.GetConfig', 'Light.GetStatus', 'Input.SetConfig',
       'Cover.GetConfig', 'Input.GetConfig', 'Light.Toggle',
       'Light.ResetCounters', 'Temperature.SetTemperature',
       'Humidity.SetConfig', 'Cover.SetConfig', 'Temperature.GetStatus',
       'Smoke.GetConfig', 'Light.GetConfig', 'Temperature.SetConfig',
       'Smoke.GetStatus', 'Cover.Calibrate', 'Switch.Toggle',
       'Cover.Close', 'Light.Calibrate', 'Humidity.GetConfig'],
      dtype=object)

In [4]:
types_environments_by_num = 6
num_locations_per_device = 10
num_device_types = 7
num_device_number = 4
num_functions_per_device = 5
print(f'{types_environments_by_num * num_locations_per_device * num_device_types * num_device_number * num_functions_per_device} possible combinations per one unique prompt')

8400 possible combinations per one unique prompt


In [5]:
600*0.8*15

7200.0